In [ ]:
import os 
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = "/kaggle/input/multi-cancer/Multi Cancer/Brain Cancer"

In [ ]:
def load_data(path):
    files = glob(os.path.join(path,"*","*"))
    return files

In [ ]:
files =load_data(path)

In [ ]:
def preprocess_data(image):
    img = cv2.imread(image,cv2.IMREAD_COLOR)
    img = cv2.resize(img,(H.W))
    img = img / 255.0
    img = img.astype(np.float32)
    return img

In [ ]:
img = preprocess_data(files[0])
img.shape